In [ ]:
%%capture
!pip install -U kaleido
!pip install -U scikit-learn
import os
os.kill(os.getpid(), 9)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
import json
import re

warnings.simplefilter('ignore')


# Funcs

In [ ]:
def read_csv(csvpath):
  df = pd.read_csv(csvpath)
  if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'],format='mixed')
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.set_index('date', inplace=True)
  elif 'date_time' in df.columns:
    df['date_time'] = pd.to_datetime(df['date_time'],format='mixed')
    df['date_time'] = df['date_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.set_index('date_time', inplace=True)
  elif 'time' in df.columns:
    df['time'] = pd.to_datetime(df['time'],format='mixed')
    df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.set_index('time', inplace=True)
  else:
    print ('cant find date index')
  print (df.columns, '\n' 'rows=', len(df))
  return df

In [ ]:
def create_savedir(root_dir, site_tofill, srs_chamber):
  timestamp = datetime.datetime.now().strftime("_%y%m%d")
  savedir = root_dir + site_tofill + '_' + srs_chamber + '_' + timestamp
  print(savedir)
  if not os.path.exists(savedir):
      os.makedirs(savedir)
  return savedir

In [ ]:
def dataset_split(dataset, SRS_CHAMBER):
  missingdata_index = dataset[dataset[SRS_CHAMBER].isnull()].index #find missing data index
  print (missingdata_index)
  available_data = dataset.drop(missingdata_index, axis = 0) # drop the missing index so the rest of the data are available for training
  print (available_data.shape)

  #split
  train = available_data.sample(frac=0.8,random_state=200)
  test = available_data.drop(train.index)
  print(train.shape,test.shape)

  y_train = train.filter([SRS_CHAMBER]).copy()
  print (y_train.shape) # training input features

  x_train = train.drop([SRS_CHAMBER], axis=1)
  print (x_train.shape) # training target

  y_test = test.filter([SRS_CHAMBER]).copy()
  print (y_test.shape) # testing input features

  x_test = test.drop([SRS_CHAMBER], axis=1)
  print (x_test.shape) # testing target

  x_predict = dataset[dataset.index.isin(missingdata_index)].drop([SRS_CHAMBER], axis=1)
  print (x_predict.shape)

  return x_train, y_train, x_test, y_test, x_predict

In [ ]:
def count_by_seasons(df):
  total = len(df)
  df.index = pd.to_datetime(df.index,format='mixed')
  df['month'] = df.index.month
  spring = df.query('month == 3').shape[0] + df.query('month == 4').shape[0] + df.query('month == 5').shape[0]
  summer = df.query('month == 6').shape[0] + df.query('month == 7').shape[0] + df.query('month == 8').shape[0]
  autumn = df.query('month == 9').shape[0] + df.query('month == 10').shape[0] + df.query('month == 11').shape[0]
  winter = df.query('month == 12').shape[0] + df.query('month == 1').shape[0] + df.query('month == 2').shape[0]
  assert total == spring + summer + autumn + winter, 'numbers not match'
  return spring, summer, autumn, winter

In [ ]:
# use regex to find SRS CHAMBER LIST
def get_srs_chambers(site_to_fill):
  site_df = read_csv(site_dict[site_to_fill])
  r = re.compile("^FD.*_Flux_Median$", re.IGNORECASE) #case insensitive
  srs_chambers_list = list(filter(r.match, list(site_df.columns)))
  print(srs_chambers_list)
  return site_df, srs_chambers_list

# import data

In [ ]:
# Paths to the time series CSV data
hcb_csv = r'/content/drive/Shareddrives/Gap Filling/hess_creek_burned_era5_merra2.csv'
hcu_csv = r'/content/drive/Shareddrives/Gap Filling/hess_creek_unburned_era5_merra2.csv'
akb_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_AKB_SRSData_2017_2019_nongapfilled.csv'
aku_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_AKU_SRSData_2017_2018_nongapfilled.csv'
atq_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_ATQ_SRSData_2017_2018_nongapfilled.csv'
bnz_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_BNZ_SRSData_2016_2023_nongapfilled.csv'
eml_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_EML_SRSData_2016_2023_nongapfilled.csv'
imh_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_IMH_SRSData_2016_2016_nongapfilled.csv'
iml_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_IML_SRSData_2016_2017_nongapfilled.csv'
ncb_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_NCB_SRSData_2017_2018_nongapfilled.csv'
ncu_csv = r'/content/drive/Shareddrives/Gap Filling/Alaska_NCU_SRSData_2016_2017_nongapfilled.csv'
era5_csv = r'/content/drive/Shareddrives/Gap Filling/era5_corrected_fdc.csv'

In [ ]:
site_dict = {'hcb':hcb_csv, 'hcu':hcu_csv, 'akb':akb_csv, 'aku':aku_csv, 'atq':atq_csv, 'bnz':bnz_csv, 'eml':eml_csv, 'imh':imh_csv, 'iml':iml_csv, 'Nome_Creek_Burned':ncb_csv, 'Nome_Creek_Unburned':ncu_csv}
era5_features = ['st2', 'le', 'pres', 'h', 'rad', 'airt', 'ppt.roll', 'rh', 'vpd', 'ws']
site_features = ['sm1','filledTSOIL1','filledTSOIL2']


era5_df = read_csv(era5_csv)

Index(['Site_Code', 'Site_Name', 'st1', 'st2', 'le', 'pres', 'h', 'rad',
       'airt', 'ppt', 'ppt.roll', 'rh', 'vpd', 'ws'],
      dtype='object') 
rows= 403224


# Batch run

In [ ]:
#model2: era5 model
ROOT_DIR = r'/content/drive/Shareddrives/Gap Filling/Filled_result/era5_model/'

sites = ['akb','aku','atq','bnz','eml','imh','iml','ncb','ncu']
features = era5_features

In [ ]:
#batch process:

for site_to_fill in sites:
  site_df, srs_chambers = get_srs_chambers(site_to_fill)
  for srs_chamber in srs_chambers:
    print (srs_chamber)
    flux_df = site_df[[srs_chamber]].copy(deep=True)

    #sitelvl features
    #feature_df = site_df[features].copy(deep=True)

    #era5 features
    era5_site_df = era5_df[era5_df['Site_Code'] == site_to_fill.upper()]
    feature_df = era5_site_df[features].copy(deep=True)

    dataset = flux_df.merge(feature_df, left_index=True, right_index=True)
    x_train, y_train, x_test, y_test, x_predict = dataset_split(dataset, srs_chamber)

    best_params = {'n_estimators': 1600,
    'min_samples_split': 5,
    'min_samples_leaf': 1,
    'max_depth': 90,
    'bootstrap': True}

    # train
    rf = RandomForestRegressor(**best_params)
    rf.fit(x_train, y_train)

    # model r2 score
    r2 = rf.score(x_test, y_test).round(3)
    print('R2:', r2)

    # slope check
    x_test_predict = rf.predict(x_test)
    slope_check = pd.DataFrame({'x': x_test_predict, 'y': y_test[srs_chamber]})
    fig = px.scatter(slope_check, x='x', y='y', trendline="ols")
    results = px.get_trendline_results(fig)
    slope = results.iloc[0]["px_fit_results"].params[1].round(3)
    print ('slope =', slope)

    # inference
    model_inference = rf.predict(x_predict)
    data_filled = srs_chamber + '_filled'
    # add one dimension as place holder to match the shape of the original data
    missingdata_index = dataset[dataset[srs_chamber].isnull()].index
    model_inference_reshape = np.expand_dims(model_inference, axis=1)
    print (model_inference_reshape.shape)
    filled_series = pd.DataFrame(model_inference_reshape).rename(columns={0:data_filled})
    filled_series.index = missingdata_index

    original_data = dataset.filter([srs_chamber]).copy(deep=True) # original data
    original_data.loc[:] = np.nan # set all values to NaN only keep the shape
    original_data.loc[missingdata_index] = filled_series # set the missing index to the filled values
    filled_data_original_masked = original_data.rename(columns={srs_chamber:data_filled})

    fig = go.Figure()

    # Plot
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=filled_data_original_masked.reset_index()['date'],
            y=filled_data_original_masked.reset_index()[data_filled],
            marker=dict(
                color='red',
                size=2,
                opacity=0.5,
            ),
            showlegend=False
        )
    )
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=filled_data_original_masked.reset_index()['date'],
            y=dataset.reset_index()[srs_chamber],
            marker=dict(
                color='blue',
                size=2,
                opacity=0.5,
            ),
            showlegend=False
        )
    )
    fig.show()

    #Metrics
    rmse = root_mean_squared_error(y_test, x_test_predict)
    n_rmse = rmse/np.mean(y_test)
    mae = mean_absolute_error(y_test, x_test_predict)
    print (rmse, '\n', n_rmse, '\n', mae)

    save_dir = create_savedir(ROOT_DIR, site_to_fill, srs_chamber)
    # create output csv
    filled_gap = filled_data_original_masked.reset_index()[data_filled]
    original = dataset.reset_index()[srs_chamber]
    output = pd.concat([original, filled_gap], axis=1)

    # save to file
    filename = data_filled + 'slope' + str(slope) + '_rs' + str(r2) + '.csv'
    print (filename)
    output.to_csv(save_dir + '/' + filename)
    figname = data_filled + 'slope' + str(slope) + '_rs' + str(r2) + '.jpeg'
    fig.write_image(save_dir + '/' + figname)

    # counting rows
    n_obs_spring, n_obs_summer, n_obs_autumn, n_obs_winter = count_by_seasons(flux_df.dropna())
    n_filled_spring, n_filled_summer, n_filled_autumn, n_filled_winter = count_by_seasons(x_predict)

    # output log
    output_log =  {'MODEL' : 'Random Forest',
               'SITE' : site_to_fill,
               'SRS_CHAMBER' : srs_chamber,
               'FEATURES' : features,
               'RSQUARE' : r2,
               'SLOPE' : slope,
               'RMSE' : rmse,
               'N_RMSE' : n_rmse,
               'MAE' : mae,
               'start_t' : flux_df.index[0],
               'end_t' : flux_df.index[-1],
               'n_filled' : len(x_predict),
               'n_observed' : len(flux_df) - len(x_predict),
               'n_obs_spring' : n_obs_spring,
               'n_obs_summer' : n_obs_summer,
               'n_obs_autumn' : n_obs_autumn,
               'n_obs_winter' : n_obs_winter,
               'n_filled_spring' : n_filled_spring,
               'n_filled_summer' : n_filled_summer,
               'n_filled_autumn' : n_filled_autumn,
               'n_filled_winter' : n_filled_winter
         }
    output_log

    #save parameters
    with open(os.path.join(save_dir, 'output_log.json'), 'w') as fp:
        json.dump(output_log, fp)
    print ('done')

Index(['FD22_flux', 'FD23_flux', 'FD24_flux', 'FD22_flux_median',
       'FD23_flux_median', 'FD24_flux_median', 'Soil_Chamber_CO2_conc_5cm',
       'Soil_Chamber_CO2_conc_15cm', 'Soil_Chamber_CO2_conc_25cm',
       'Soil_Chamber_TEMP_5cm', 'Soil_Chamber_TEMP_15cm',
       'Soil_Chamber_TEMP_25cm', 'Soil_Chamber_TEMP_ATM', 'SoilTemp15cm_01',
       'SoilTemp15cm_02', 'VWC15cm_Raw_01', 'VWC15cm_Raw_02',
       'VWC15cm_CALI_01', 'VWC15cm_CALI_02'],
      dtype='object') 
rows= 19171
['FD22_flux_median', 'FD23_flux_median', 'FD24_flux_median']
FD22_flux_median
Index(['2017-06-23 17:00:00', '2017-06-23 18:00:00', '2017-06-23 19:00:00',
       '2017-06-23 20:00:00', '2017-06-23 21:00:00', '2017-06-23 22:00:00',
       '2017-06-23 23:00:00', '2017-06-24 01:00:00', '2017-06-24 02:00:00',
       '2017-06-24 03:00:00',
       ...
       '2019-08-26 10:00:00', '2019-08-28 03:00:00', '2019-08-28 04:00:00',
       '2019-08-28 06:00:00', '2019-08-28 07:00:00', '2019-08-30 07:00:00',
       '2019-0

ValueError: Length mismatch: Expected axis has 13114 elements, new values have 13113 elements

# Unit testing

In [ ]:
ROOT_DIR = r'/content/drive/Shareddrives/Gap Filling/Filled_result/test/'

site_tofill = 'akb'
features = era5_features

site_df = read_csv(site_dict[site_tofill])
r = re.compile("^FD.*_Flux$", re.IGNORECASE) #case insensitive
srs_chambers_list = list(filter(r.match, list(site_df.columns)))
print(srs_chambers_list)

Index(['FD22_flux', 'FD23_flux', 'FD24_flux', 'FD22_flux_median',
       'FD23_flux_median', 'FD24_flux_median', 'Soil_Chamber_CO2_conc_5cm',
       'Soil_Chamber_CO2_conc_15cm', 'Soil_Chamber_CO2_conc_25cm',
       'Soil_Chamber_TEMP_5cm', 'Soil_Chamber_TEMP_15cm',
       'Soil_Chamber_TEMP_25cm', 'Soil_Chamber_TEMP_ATM', 'SoilTemp15cm_01',
       'SoilTemp15cm_02', 'VWC15cm_Raw_01', 'VWC15cm_Raw_02',
       'VWC15cm_CALI_01', 'VWC15cm_CALI_02'],
      dtype='object') 
rows= 19171
['FD22_flux', 'FD23_flux', 'FD24_flux']


In [ ]:
SRS_CHAMBER = 'FD22_flux'

In [ ]:
flux_df = site_df[[SRS_CHAMBER]].copy(deep=True)

#sitelvl features
#feature_df = site_df[features].copy(deep=True)

#era5 features
era5_site_df = era5_df[era5_df['Site_Code'] == site_tofill.upper()]
feature_df = era5_site_df[features].copy(deep=True)

dataset = flux_df.merge(feature_df, left_index=True, right_index=True)
x_train, y_train, x_test, y_test, x_predict = dataset_split(dataset, SRS_CHAMBER)

Index(['2017-06-23 17:00:00', '2017-06-23 18:00:00', '2017-06-23 19:00:00',
       '2017-06-23 20:00:00', '2017-06-23 21:00:00', '2017-06-23 22:00:00',
       '2017-06-23 23:00:00', '2017-06-24 00:00:00', '2017-06-24 01:00:00',
       '2017-06-24 02:00:00',
       ...
       '2019-08-26 07:00:00', '2019-08-26 08:00:00', '2019-08-26 10:00:00',
       '2019-08-28 03:00:00', '2019-08-28 04:00:00', '2019-08-28 06:00:00',
       '2019-08-28 07:00:00', '2019-08-30 07:00:00', '2019-08-30 08:00:00',
       '2019-08-31 07:00:00'],
      dtype='object', length=13679)
(5491, 11)
(4393, 11) (1098, 11)
(4393, 1)
(4393, 10)
(1098, 1)
(1098, 10)
(13680, 10)


In [ ]:
print (len(dataset))
missingdata_index = dataset[dataset[SRS_CHAMBER].isnull()].index #find missing data index
print (missingdata_index)
available_data = dataset.drop(missingdata_index, axis = 0) # drop the missing index so the rest of the data are available for training
print (available_data.shape)

x_predict = dataset[dataset.index.isin(missingdata_index)].drop([SRS_CHAMBER], axis=1)
print (x_predict.shape)

19171
Index(['2017-06-23 17:00:00', '2017-06-23 18:00:00', '2017-06-23 19:00:00',
       '2017-06-23 20:00:00', '2017-06-23 21:00:00', '2017-06-23 22:00:00',
       '2017-06-23 23:00:00', '2017-06-24 00:00:00', '2017-06-24 01:00:00',
       '2017-06-24 02:00:00',
       ...
       '2019-08-26 07:00:00', '2019-08-26 08:00:00', '2019-08-26 10:00:00',
       '2019-08-28 03:00:00', '2019-08-28 04:00:00', '2019-08-28 06:00:00',
       '2019-08-28 07:00:00', '2019-08-30 07:00:00', '2019-08-30 08:00:00',
       '2019-08-31 07:00:00'],
      dtype='object', length=13679)
(5491, 11)
(13680, 10)


In [ ]:
best_params = {'n_estimators': 1600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_depth': 90,
 'bootstrap': True}

# train
rf = RandomForestRegressor(**best_params)
rf.fit(x_train, y_train)

# model r2 score
r2 = rf.score(x_test, y_test).round(3)
print('R2:', r2)

R2: 0.926


In [ ]:
# slope check
x_test_predict = rf.predict(x_test)
slope_check = pd.DataFrame({'x': x_test_predict, 'y': y_test[SRS_CHAMBER]})
fig = px.scatter(slope_check, x='x', y='y', trendline="ols")
results = px.get_trendline_results(fig)
slope = results.iloc[0]["px_fit_results"].params[1].round(3)
print ('slope =', slope)

slope = 1.005


In [ ]:
# inference
model_inference = rf.predict(x_predict)
data_filled = SRS_CHAMBER + '_filled'
# add one dimension as place holder to match the shape of the original data
missingdata_index = dataset[dataset[SRS_CHAMBER].isnull()].index
model_inference_reshape = np.expand_dims(model_inference, axis=1)
print (model_inference_reshape.shape)
filled_series = pd.DataFrame(model_inference_reshape).rename(columns={0:data_filled})
filled_series.index = missingdata_index

original_data = dataset.filter([SRS_CHAMBER]).copy(deep=True) # original data
original_data.loc[:] = np.nan # set all values to NaN only keep the shape
original_data.loc[missingdata_index] = filled_series # set the missing index to the filled values
filled_data_original_masked = original_data.rename(columns={SRS_CHAMBER:data_filled})

(13114, 1)


ValueError: Length mismatch: Expected axis has 13114 elements, new values have 13113 elements

In [ ]:
fig = go.Figure()

#filled data with original data masked
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=filled_data_original_masked.reset_index()['date'],
        y=filled_data_original_masked.reset_index()[data_filled],
        marker=dict(
            color='red',
            size=2,
            opacity=0.5,
        ),
        showlegend=False
    )
)

#original data
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=filled_data_original_masked.reset_index()['date'],
        y=dataset.reset_index()[SRS_CHAMBER],
        marker=dict(
            color='blue',
            size=2,
            opacity=0.5,
        ),
        showlegend=False
    )
)

fig.show()

In [ ]:
rmse = root_mean_squared_error(y_test, x_test_predict)
n_rmse = rmse/np.mean(y_test)
mae = mean_absolute_error(y_test, x_test_predict)
print (rmse, '\n', n_rmse, '\n', mae)

0.27220797030489463 
 0.2456610454947448 
 0.17085079683610555


In [ ]:
save_dir = create_savedir(ROOT_DIR, SITE_TOFILL, SRS_CHAMBER)
# create output csv
filled_gap = filled_data_original_masked.reset_index()[data_filled]
original = dataset.reset_index()[SRS_CHAMBER]
output = pd.concat([original, filled_gap], axis=1)

# save to file
filename = data_filled + 'slope' + str(slope) + '_rs' + str(r2) + '.csv'
print (filename)

output.to_csv(save_dir + '/' + filename)

figname = data_filled + 'slope' + str(slope) + '_rs' + str(r2) + '.jpeg'
fig.write_image(save_dir + '/' + figname)

/content/drive/Shareddrives/Gap Filling/Filled_result/hcu_FD01_Flux_Median_era5_240410_1732
FD01_Flux_Median_filledslope1.005_rs0.832.csv


In [ ]:
#spring (March, April, May), summer (June, July, August), autumn (September, October, November), winter (December, Jan, Feb)
n_obs_spring, n_obs_summer, n_obs_autumn, n_obs_winter = count_by_seasons(flux_df.dropna())
n_filled_spring, n_filled_summer, n_filled_autumn, n_filled_winter = count_by_seasons(x_predict)

In [ ]:
output_log =  {'MODEL' : 'Random Forest',
               'SITE' : SITE_TOFILL,
               'SRS_CHAMBER' : SRS_CHAMBER,
               'FEATURES' : FEATURES,
               'RSQUARE' : r2,
               'SLOPE' : slope,
               'RMSE' : rmse,
               'N_RMSE' : n_rmse,
               'MAE' : mae,
               'start_t' : flux_df.index[0],
               'end_t' : flux_df.index[-1],
               'n_filled' : len(x_predict),
               'n_observed' : len(flux_df) - len(x_predict),
               'n_obs_spring' : n_obs_spring,
               'n_obs_summer' : n_obs_summer,
               'n_obs_autumn' : n_obs_autumn,
               'n_obs_winter' : n_obs_winter,
               'n_filled_spring' : n_filled_spring,
               'n_filled_summer' : n_filled_summer,
               'n_filled_autumn' : n_filled_autumn,
               'n_filled_winter' : n_filled_winter
         }
output_log

#save parameters
with open(os.path.join(save_dir, 'output_log.json'), 'w') as fp:
    json.dump(output_log, fp)
print ('done')

done
